# Call Center Instrumentation & Analytics (CCIA)

## Watson-Call-Center-Think18 Lab March 2015

This document provides guidance and background for a hands-on Python + IBM Watson lab being presented at an IBM Think2018 conference in March 2018, and for an IPython / Jupyter notebook and python code available for open source after the event.

The focus is Call Center Instrumentation and Analytics (CCIA) pattern.   The Notebook and information here seek to help organizations beginning to explore how to better understand the unstructured "dark data" that arises from phone calls to call centers. 

### Why is this useful?
Enterprises spend more than $1 trillion on 250 billion customer service calls each year.  By using multiple IBM Watson "signal services" to extract signal from raw audio data; perform data analytics, clustering, unsupervised and machine learning, and visualizations, technical teams can use data understand patterns in call centers. KPI and ROI positive.

### What is the process? And what Watson services are used?
Step 1 - Speech to Text (STT) – Converts Raw Audio to Transcripts; 
Step 2 - Natural Language Understanding (NLU) - extracts features concepts, entities, keywords, categories/topics, sentiment and emotion; 
Step 3 - Natural Language Classifier (NLC) - is a user trained classification service, with user defined “ground truth” that classifies text chunks; 
Step 4 - Tone Analyzer (Tone) – uses linguistic analysis to detect emotional and language tones in written text; 
Step 5 - Call Center Analytics – analyzes and visualizes the data signal to allow for interpretation of data and in cases, actionable insights; 


### Beginner Audience & Focus on Basics
•	This is a beginner lab intended to educate on the fundamentals of getting from data to insights with IBM Watson and open source tools 
•	Audience may include IT and operations teams curious about enriching unstructured data – the lab is NOT intended for sophisticated call center technologists 
•	Lab/code does NOT purport to compete with expensive and sophisticated solutions already in market 
•	The lab and code cover the basics – to educate on the fundamental plumbing and steps, to provide base for instrumentation 

### Success Metrics
If successful – the lab participants or notebook users will
1.	Gain experience in using an IPython / Jupyter notebook
https://ipython.org/notebook.html
2.	Connect to four Watson Developer Cloud ‘signal service’ APIs 
https://www.ibm.com/watson/developer/ 
3.	Connect to IBM Cloud Object storage for data read and write 
https://www.ibm.com/cloud/object-storage 
4.	Understand whether/how the tools and methods might benefit org
https://github.com/mamoonraja/call-center-think18/tree/master/notebooks


## Notebook 1 – Speech to Text (STT) & First Contact
## Install Python Dependencies

Python’s standard library is very extensive, offering a wide range of facilities.  It contains built-in modules like JSON a lightweight data interchange format.  https://docs.python.org/2/library/index.html and https://docs.python.org/2/library/json.html

IBM Watson Developer Cloud has a Python client library to quickly get started with the various Watson APIs services.
https://pypi.python.org/pypi/watson-developer-cloud

Using Python with IBM COS: Python support is provided through the Boto 3 library. The boto3 library provides complete access and can source credentials. The IBM COS endpoint must be specified when creating a service resource or low-level client as shown in documentation
https://ibm-public-cos.github.io/crs-docs/python


In [14]:
#imports.... Run this each time after restarting the Kernel
#!pip install watson_developer_cloud
import watson_developer_cloud as watson
import json
from botocore.client import Config
import ibm_boto3


### Add COS and STT Credentials

Copy paste the following snippet to next cell, and add your own set of credentials  

For Cloud Object Storage

```code
credentials_os = {
    'IBM_API_KEY_ID': '',
    'IAM_SERVICE_ID': '',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': '',
}
```

For IBM Watson Speech to Text (STT) API

```code
credentials_stt = {
    "url": "",
    "username": "",
    "password": ""
}

```

## Getting COS Credentials 
IBM Cloud Object Storage is a highly scalable cloud storage service, designed for high durability, resiliency and security. Store, manage and access your data via our self-service portal and RESTful APIs. Connect applications directly to Cloud Object Storage use other IBM Cloud Services with your data.
Create LITE (free) service here https://console.bluemix.net/catalog/services/cloud-object-storage and get credentials by clicking on service credentials and then “view credential” 


## Getting STT Credentials 
Importing Credentials – Each Watson signal service (STT, NLC, NLU and Tone) will require credentials - a username and password
If you already have an IBM Cloud / Bluemix account login here https://console.bluemix.net/ but if you have not yet registered for IBM Cloud - you will need to Register for a Free account here https://www.ibm.com/watson/developer/ registration takes less than 4 minutes and is free. More information here https://www.ibm.com/watson/developer-resources/ 

Once logged in - go to https://console.bluemix.net/developer/watson/dashboard - browse services for SPEECH TO TEXT, and select Details, Create service from here https://console.bluemix.net/catalog/services/speech-to-text  - for free you can select LITE Plan 
LITE plan for STT “gets you started with 100 minutes per month at no cost”

The Username and Password (and URL) is found by clicking on service credentials and then “view credential” 


In [15]:
# The code was removed by DSX for sharing.

In [16]:
client = ibm_boto3.client(
    service_name = 's3', 
    ibm_api_key_id = credentials_os['IBM_API_KEY_ID'],
    ibm_auth_endpoint = credentials_os['IBM_AUTH_ENDPOINT'],
    config = Config(signature_version = 'oauth'),
    endpoint_url = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'
)


## Speect to Text 

Following cell has two methods:
 - `get_transcript()` calls speech to text enpoint and generates a text transcript for you for a sample audio file.
 - `analyze_sample()` gets the sample object from cloud storage, calls get_transcript to fetch the tranccript, and saves your transcript in cloud storage as `<file_name>_text.json`.
 
OGG, WAV FLAC, L16, MP3, MPEG formats are options for the IBM Watson STT service.  For the lab we use OGG samples. 

In [17]:
#STT

import json
import io
from os.path import join, dirname
from watson_developer_cloud import SpeechToTextV1

speech_to_text = SpeechToTextV1(
    username = credentials_stt['username'],
    password = credentials_stt['password'],
    x_watson_learning_opt_out=False
)


# OGG, WAV FLAC, L16, MP3, MPEG formats are options for the STT service 
# with Narrowband (generaly telco) and Broadband (e.g. higher quality USB mic) audio.  
# For the LAB – OGG format was used for sample files in lab. Of other audio formats e.g. WAV - remember to change 'OGG' content_type='audio/ogg' in code below if you do.

#get transcript Very basic one
def get_transcript(audio):
    transcript = json.dumps(speech_to_text.recognize(audio, content_type='audio/ogg', timestamps=True,
        word_confidence=True), indent=2)
    return transcript

def analyze_sample(sample):
    streaming_body = client.get_object(Bucket = credentials_os['BUCKET'], Key=sample)['Body']
    text = get_transcript(streaming_body.read())
    client.put_object(Bucket = credentials_os['BUCKET'], Key = sample.split('.')[0] + '_text.json', Body = text)
    return text



## Analyze the selected audio files


In [18]:
file_list = ['sample1-addresschange-positive.ogg',
             'sample2-address-negative.ogg',
             'sample3-shirt-return-weather-chitchat.ogg',
             'sample4-angryblender-sportschitchat-recovery.ogg',
             'sample5-calibration-toneandcontext.ogg',
             'jfk_1961_0525_speech_to_put_man_on_moon.ogg',
             'May 1 1969 Fred Rogers testifies before the Senate Subcommittee on Communications.ogg']

# jfk_1961_0525_speech_to_put_man_on_moon.ogg  new and longer
# May 1 1969 Fred Rogers testifies before the Senate Subcommittee on Communications.ogg


# FILE LIST – in this notebook, each OGG file produces its own transcript.  These ones are quite short so it happens quickly.  
# Longer (e.g. 1 hour) audio files may justify using asynchronous method, and a real time a sessions method (both defined below)

# For longer files and transcription at scale: https://www.ibm.com/watson/developercloud/speech-to-text/api/v1/  
# WebSockets includes a single method that establishes a persistent connection with the service over the WebSocket protocol.
# Sessionless includes a method that provides a simple means of transcribing audio without the overhead of establishing and maintaining a session.
# Sessions provides methods that allow a client to maintain a long, multi-turn exchange, or session, with the service or to establish multiple parallel conversations with a particular instance of the service.
# Asynchronous provides a non-blocking interface for transcribing audio. You can register a callback URL to be notified of job status and, optionally, results, or you can poll the service to learn job status and retrieve results manually.


In [ ]:
# TRANSCRIBE – this is where STT receives the OGG files provided and returns text to TRANSCRIPT
# this is a test of ONE transcription in the list - place '0' - may take a minute
transcript = analyze_sample(file_list[0])

for result in json.loads(transcript)['results']:
    print(result['alternatives'][0]['transcript'], result['alternatives'][0]['confidence'])

good morning  0.995
can you give me some help I'd like to change my address please  0.999
my name is Ryan Smith  0.972
I am from Sacramento California  0.969
that's right  0.996
my phone number is five five five one two one two  0.839
yes that's me  0.997
my old address is number one two three oak street  0.965
my new address is five six seven pine street  0.869
yes and the zip is nine zero two one zero  0.921
yep that's right  0.89
now the phone number stays the same  0.808
that's right I would like to keep all the options of said no other changes the only thing that I want to change is the address  0.944
yes that's right  0.995
yep  0.583
very good yes thank you so much for help  0.921
it  0.419
thanks have a good day bye bye  0.994


In [ ]:
# Testing position [6] - which is the LONGER OGG file - 7th position Fred Rogers - just under 7 minutes

transcript = analyze_sample(file_list[6])

for result in json.loads(transcript)['results']:
    print(result['alternatives'][0]['transcript'], result['alternatives'][0]['confidence'])

In [37]:
### analyze more samples and display results in better format (as table)?